In [29]:
import os
import json
import glob

import numpy as np
import pandas as pd

from tti_explorer.strategies.common import RETURN_KEYS
from tti_explorer.utils import LatexTableDeck

# import sys; sys.path.append("../scripts/charts")
# from chart_books import *

In [30]:

columns = [
    RETURN_KEYS.stopped_by_social_distancing_percentage,
    RETURN_KEYS.stopped_by_symptom_isolating_percentage,
    RETURN_KEYS.stopped_by_tracing_percentage,
    RETURN_KEYS.not_stopped_asymptomatic_percentage,
    RETURN_KEYS.not_stopped_symptomatic_non_compliant_percentage,
    RETURN_KEYS.not_stopped_by_tti_percentage,
]

columns_pretty_names = [
    "Prevented by social distancing",
    "Prevented by isolation of index case at onset of symptoms (pre-test result) and quarantine of household contacts",
    "Prevented by tracing and quarantine of extra-household secondary contacts",
    "Transmissions from asymptomatic index cases",
    "Transmissions from never-reporting symptomatic index cases",
    "Transmissions from reporting index cases",
]

columns_order = [
    "Transmissions from asymptomatic index cases",
    "Transmissions from never-reporting symptomatic index cases",
    "Transmissions from reporting index cases",
    "Prevented by tracing and quarantine of extra-household secondary contacts",
    "Prevented by isolation of index case at onset of symptoms (pre-test result) and quarantine of household contacts",
    "Prevented by social distancing",
][::-1]


def get_caption(name, col_names):
    cap = r"""
    {name}
    \begin{{enumerate}}[a)]
    {inner}
    \end{{enumerate}}
    """
    inner = ""
    for col in col_names:
        inner += f"\t \item {col}\n"
    return cap.format(name=name, inner=inner.strip())


rename_dct = dict((name, f"({chr(97+i)})") for i, name in enumerate(columns_order))



In [31]:
def load_sensitivity_folder(outputs_dir, config_keys, config_pretty_names):
    sensitivity_data = pd.DataFrame()
    indices = [int(k.replace(".json", "").replace("config_", "")) for k in os.listdir(outputs_dir)
              if k.startswith("config_")]
            
    for i in indices:
        with open(os.path.join(outputs_dir, f'config_{i}.json'), 'r') as file:
            cfg = json.load(file)
        results = pd.read_csv(os.path.join(outputs_dir, f'run_{i}.csv'))
        for key, name in zip(config_keys, config_pretty_names):
            results[name] = cfg["config"][key]
        # print(cfg["config"][config_key])
        sensitivity_data = pd.concat([sensitivity_data, results])
    sensitivity_data = sensitivity_data.round(2)
    sensitivity_data.set_index([*config_pretty_names], inplace=True)

    # change % to R
    return sensitivity_data

In [32]:
subtable_template = r"""
\begin{subtable}[H]{0.48\textwidth}
        %(table)s
        \caption{%(caption)s}
    \end{subtable}
    """.strip()


key_table = r"""
\begin{center}
\begin{tabularx}{\textwidth}{c>{\small}X}
\toprule
\multicolumn{2}{c}{Key} \\
\midrule
"""
for i, name in enumerate(columns_order):
    key_table += f"({chr(97+i)}) & {name} \\\ \n"
#     (a) & Transmissions from asymptomatic index cases \\
#     (b) & Transmissions from never-reporting symptomatic index cases \\
#     (c) & Transmissions from reporting index cases \\
#     (d) & Prevented by tracing and quarantine of extra-household secondary contacts \\
#     (e) & Prevented by isolation of index case at onset of symptoms (pre-test result) and quarantine of household contacts \\
#     (f) & Prevented by social distancing \\
key_table +=r"""
\bottomrule
\end{tabularx}
\end{center}"""

In [33]:
folder = "/Users/bryn/Programming/repos/tti-explorer/outputs/from-mike"

levels = [f"S{i}" for i in range(1, 6)]

for level in levels:
    odir = "{name}/{level}_test_based_TTI"

    app_uptake_sensitivity = load_sensitivity_folder(
        outputs_dir = os.path.join(folder, "app_cov", f"{level}_test_based_TTI"),
        config_keys  = ["app_cov"] ,
        config_pretty_names = ["Application uptake"],
    )
    
    app_table = format_table(
    app_uptake_sensitivity,
        add_stds=False
    ).rename(index=lambda x: f"${x*100:.0f}\\%$", columns=rename_dct)
    app_table.index.name = "App Uptake"


    trace_adherance_sensitivity = load_sensitivity_folder(
        outputs_dir = os.path.join(folder, "trace_adherence", f"{level}_test_based_TTI"),
        config_keys  = ["trace_adherence"] ,
        config_pretty_names = ["Policy Adherence"],
    )
    
    compliance_table = format_table(
    trace_adherance_sensitivity,
    add_stds=False
    ).rename(index=lambda x: f"${x*100:.0f}\\%$", columns=rename_dct)
    compliance_table.index.name = "Compliance"


    delay_sensitivity = load_sensitivity_folder(
        outputs_dir = os.path.join(folder, "delays", f"{level}_test_based_TTI"),
        config_keys  = ["testing_delay", "manual_trace_delay"] ,
        config_pretty_names = ["Testing Delay", "Manual Tracing Delay"],
    )
    
    tt_idx_order = [(3, 3), (3, 2), (2, 3), (3, 1), (2, 2), (1, 3), (2, 1), (1, 2), (1, 1)]

    delay_table = format_table(
        delay_sensitivity,
        add_stds=False
    ).loc[tt_idx_order].rename(index=lambda x: "{}/{}".format(*x), columns=rename_dct)
    delay_table.index.name = "Delay"
    
    deck = LatexTableDeck()
    deck.add_package("enumitem", options=['shortlabels'])
    deck.add_package("fullpage")
    deck.add_package("geometry", options=["margin=3cm"])
    deck.add_package("subcaption")
    
    with open(f"{level}-app-uptake.tex", "w") as f:
        f.write(app_table.to_latex(escape=False))
    with open(f"{level}-compliance.tex", "w") as f:
        f.write(compliance_table.to_latex(escape=False))
    with open(f"{level}-delay.tex", "w") as f:
        f.write(delay_table.to_latex(escape=False))
        
        
    subtab_strs = [
        r"\hspace{-2em}",
            subtable_template % {'table': app_table.to_latex(escape=False), 'caption': "Application Uptake"},

        r"\hspace{2em}",
        subtable_template % {'table': compliance_table.to_latex(escape=False), 'caption': "Compliance"},
        r"\\ " + r"\vspace{2em}",
        r"\hspace{-2em}",

            subtable_template % {'table': delay_table.to_latex(escape=False), 'caption': "Test/Trace Delays"},
        r"\hspace{2em}",
        subtable_template % {'table': key_table, 'caption': "Column Names"},
    ]
    deck.add_table("\n".join(subtab_strs), caption="")

    deck.make(os.path.join(folder, "tables", f"{level}-subtables.tex"))
    

In [28]:
def get_stat(full, stat):
    tab = full.query(f"statistic=='{stat}'")
    output= tab[columns].rename(dict(zip(columns, columns_pretty_names)), axis=1)[columns_order]
    output.index = output.index.tolist()
    return output.sort_index()


def format_table(df, add_stds, fmt_func=lambda x: f"{x:.0f}\\%"):
    table = get_stat(df, 'mean').applymap(fmt_func)
    if add_stds:
        stds = get_stat(df, 'std')
        
        table = table.add(
                " \pm "
            ).add(
                stds.mul(
                    1.96 
                ).applymap(fmt_func)
            )
    return table.applymap(lambda x: f"${x}$")

In [18]:
deck = LatexTableDeck()

In [19]:
app_table = format_table(
    app_uptake_sensitivity,
    add_stds=False
).rename(index=lambda x: f"${x*100:.0f}\\%$", columns=rename_dct)
app_table.index.name = "App Uptake"

deck.add_table(
    app_table.to_latex(escape=False),
    caption="Application Uptake"
)
app_table

UndefinedVariableError: name 'statistic' is not defined

In [8]:
compliance_table = format_table(
    trace_adherance_sensitivity,
    add_stds=False
).rename(index=lambda x: f"${x*100:.0f}\\%$", columns=rename_dct)
compliance_table.index.name = "Compliance"

deck.add_table(
    compliance_table.to_latex(escape=False),
    caption="Compliance"
)
compliance_table

,(a),(b),(c),(d),(e),(f)
Compliance,,,,,,
$50\%$,$31\%$,$17\%$,$2\%$,$6\%$,$26\%$,$17\%$
$60\%$,$31\%$,$21\%$,$3\%$,$7\%$,$21\%$,$17\%$
$70\%$,$31\%$,$25\%$,$5\%$,$7\%$,$15\%$,$17\%$
$80\%$,$31\%$,$29\%$,$6\%$,$7\%$,$10\%$,$17\%$
$90\%$,$31\%$,$33\%$,$7\%$,$7\%$,$6\%$,$17\%$
$100\%$,$31\%$,$37\%$,$9\%$,$6\%$,$0\%$,$17\%$


In [9]:
tt_idx_order = [(3, 3), (3, 2), (2, 3), (3, 1), (2, 2), (1, 3), (2, 1), (1, 2), (1, 1)]

delay_table = format_table(
    delay_sensitivity,
    add_stds=False
).loc[tt_idx_order].rename(index=lambda x: "{}/{}".format(*x), columns=rename_dct)
delay_table.index.name = "Delay"

In [10]:
delay_table = format_table(
    delay_sensitivity,
    add_stds=False
).loc[tt_idx_order].rename(index=lambda x: "{}/{}".format(*x), columns=rename_dct)
delay_table.index.name = "Delay"

deck.add_table(
    delay_table.to_latex(escape=False),
    caption="Test/Trace Delays"
)
delay_table

,(a),(b),(c),(d),(e),(f)
Delay,,,,,,
3/3,$31\%$,$29\%$,$3\%$,$11\%$,$10\%$,$17\%$
3/2,$31\%$,$29\%$,$4\%$,$10\%$,$10\%$,$17\%$
2/3,$31\%$,$29\%$,$4\%$,$10\%$,$10\%$,$17\%$
3/1,$31\%$,$29\%$,$5\%$,$9\%$,$10\%$,$17\%$
2/2,$31\%$,$29\%$,$5\%$,$8\%$,$10\%$,$17\%$
1/3,$31\%$,$29\%$,$5\%$,$8\%$,$10\%$,$17\%$
2/1,$31\%$,$29\%$,$6\%$,$7\%$,$10\%$,$17\%$
1/2,$31\%$,$29\%$,$6\%$,$7\%$,$10\%$,$17\%$
1/1,$31\%$,$29\%$,$7\%$,$6\%$,$10\%$,$17\%$


In [12]:
print(key_table)


\begin{center}
\begin{tabularx}{\textwidth}{c>{\small}X}
\toprule
\multicolumn{2}{c}{Key} \\
\midrule
(a) & Prevented by social distancing \\ 
(b) & Prevented by isolation of index case at onset of symptoms (pre-test result) and quarantine of household contacts \\ 
(c) & Prevented by tracing and quarantine of extra-household secondary contacts \\ 
(d) & Transmissions from reporting index cases \\ 
(e) & Transmissions from never-reporting symptomatic index cases \\ 
(f) & Transmissions from asymptomatic index cases \\ 

\bottomrule
\end{tabularx}
\end{center}


In [13]:
deck.add_table(key_table, caption="")

In [14]:
level = "S3"

info = get_caption("Key:", columns_order)
deck.add_string(info)

deck.add_package("enumitem", options=['shortlabels'])
deck.add_package("fullpage")
deck.add_package("geometry", options=["margin=3cm"])
deck.make(os.path.join(folder, "tables", f"{level}-tables.tex"))